In [1]:
# Imports
import os
from glob import glob
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats.qmc as qmc

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning) # for RuntimeWarning only

In [2]:
def read_INP_data(filename, conc_suffix):
    
    data = pd.read_csv(filename,index_col='temp_bin')
    # If you want to mask any of the data to remove particular runs, you need to add this here.
    # It is easy to filter so that you only get columns that e.g. contain "CAO" or have
    # heights > 2000m if they're in the column name.
    if conc_suffix == '_NT':
        err_suffix = '_err'
    else:
        err_suffix = conc_suffix+'_err'
    raw_concs = data[data.columns[pd.Series(data.columns).str.endswith(conc_suffix)]]
    raw_concs.columns = raw_concs.columns.str.replace(conc_suffix,'')
    logged_concs = np.log(raw_concs)


    raw_errors = data[data.columns[pd.Series(data.columns).str.endswith(err_suffix)]]
    raw_errors.columns = raw_errors.columns.str.replace(err_suffix,'')
    temps = logged_concs.index.values
    return temps, raw_concs, logged_concs, raw_errors

In [3]:
def four_param_fit(T, ln_nu, T_max, a, b):
    # Apply the fit to an array of temperatures T
    return ln_nu + a*(T_max-T)**b

In [4]:
def strip_nans(*args):
    # Strip nans from multiple inputs where NaN values have the same indices in each input.
    nan_mask = np.logical_not(np.isnan(args[0]))
    nanless = []
    for a in args:
        nanless.append(a[nan_mask])
    return nanless

In [5]:
def reduced_chi_squared_test(p, temps, log_concs, weighted_error):
    # Perform reduced chi-squared test for fit parameters p
    modelled = four_param_fit(temps, *p) #*p unlocks the list p so this is like writing temps, p[0], p[1]...
    residual = log_concs - modelled
    chi_sq = np.sum((residual/weighted_error)**2)
    red_chi_sq = chi_sq/(len(log_concs)-len(p))
    return red_chi_sq

In [6]:
# Create 4-dimensional latin hypercube object. Seed chosen for replication of results.
lhc = qmc.LatinHypercube(4, seed=280299)
# Take 100 LH samples
samples = lhc.random(n=100)

In [7]:
def bounded_and_sampled_scipy_4_param(temps, logged_concs, raw_concs, raw_errors, samples, run_id,
                                      nu_range=(-6,-2), plot=False, silent=True):
    """
    Ugly but functional function to find potential best-fits.
    
    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    run_id: string
        identifier of the run
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)
    plot: boolean
        not yet implemented
    silent: boolean
        say when convergence fails (likely to happen often among the 100 samples)

    Returns
    -------
    pandas
        all derived parameters for the run that converged
    """

    # Create dataframe to hold parameters for the run                                      
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    # Strip nans where appropriate as curve_fit doesn't handle them
    nanless_concs, nanless_raw, nanless_errs, nanless_temps = strip_nans(
                                                        logged_concs[run_id].values,
                                                        raw_concs[run_id].values,
                                                        raw_errors[run_id].values,
                                                        temps
    )
    # Weight errors in logarithmic space
    nanless_weights = nanless_errs/nanless_raw

    # Set bounds of the sample space
    lower_bounds = [nu_range[0], max(nanless_temps)-2, 0.3, 0.5]
    upper_bounds = [nu_range[1], max(nanless_temps)+2, 1.05, 1.0]

    # Apply the LH samples to the sample space
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)

    # Loop through all of the samples and attempt to find a best-fit
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, nanless_temps, nanless_concs, p0=ps,
                sigma=nanless_weights, bounds=(lower_bounds,upper_bounds), maxfev=20000)
            # Calculate goodness of fit parameter
            X2 = reduced_chi_squared_test(p, nanless_temps, nanless_concs, nanless_weights)
            # Store in the in the dataframe
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            # Do not fail if convergence can't be found.
            if not silent:
                print('Failed to converge with parameters', ps)
    return run_params

In [8]:
def get_parameter_df(temps, logged_concs, raw_concs, raw_errors, samples, nu_range=(-6,-2)):
    """
    Get a dataframe with the parameters that have the lowest reduced chi-squared for each run.

    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)

    Returns
    -------
    pandas Dataframe
        best parameters for each run
    """
    bounded_params = {}
    for run in logged_concs:
        # Loop through every run to get all parameters and add to dictionary
        bounded_params[run] = bounded_and_sampled_scipy_4_param(temps, logged_concs, raw_concs,
                                                                raw_errors, samples, run, nu_range)

    # create an empty dataframe to store the lowest X2 values
    lowest_X2_bounded_df = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])

    # loop through the dictionary and find the lowest X2 value for each run
    for run_id, df in bounded_params.items():
        df = df.dropna()
        if df.empty:
            # create a new row with NaN values and concatenate it to lowest_X2_df
            lowest_X2_bounded_row = pd.DataFrame([[np.nan]*len(lowest_X2_bounded_df.columns)], 
                                        columns=lowest_X2_bounded_df.columns, 
                                        index=[run_id])
            lowest_X2_bounded_df = pd.concat([lowest_X2_bounded_df, lowest_X2_bounded_row])
        else:
            lowest_X2_bounded_row = df.nsmallest(1, 'X2')
            lowest_X2_bounded_row.index = [run_id]
            lowest_X2_bounded_df = pd.concat([lowest_X2_bounded_df, lowest_X2_bounded_row])

    return lowest_X2_bounded_df

In [11]:
base = '/home/users/erinraif/acao_data/inp_data/'
suffixes = ['_NT', '_nN', '_nS', '_nV']
INP_concs = 'INP_concentrations.csv'
nX_vals = 'INP_normalised_by_aerosol.csv'

In [12]:
files = [INP_concs, nX_vals, nX_vals, nX_vals]
nu_range = [(-6,-2), (-18,-13), (12, 17), (25,33)]

In [13]:
params_df = {}
for s, csv, nus in zip(suffixes, files, nu_range):
    temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+csv, s)
    params = get_parameter_df(temps, logged_concs, raw_concs, raw_errors, samples, nus)
    params_df[s[1:]] = params

In [14]:
params_df['n_INP'] = params_df.pop('NT')


In [15]:
dfs_with_suffixes = []

# Loop through the dictionary and add suffixes to the columns
for key, df in params_df.items():
    df = df.add_suffix(f'_{key}')
    dfs_with_suffixes.append(df)

export_params = pd.concat(dfs_with_suffixes, axis=1)

In [16]:
export_params.index.name = 'unique_ID'
export_params = export_params.loc[:, ~export_params.columns.str.contains('X2|cov')]


In [17]:
export_params.to_csv('/home/users/erinraif/acao_data/metadata/parametrisations_final.csv')

## Median INP and nS parametrisations

In [20]:
def median_4_param(temps, logged_concs, raw_concs, samples, run_id,
                                      nu_range=(-6,-2), plot=False, silent=True):
    """
    Ugly but functional function to find potential best-fits.
    
    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    run_id: string
        identifier of the run
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)
    plot: boolean
        not yet implemented
    silent: boolean
        say when convergence fails (likely to happen often among the 100 samples)

    Returns
    -------
    pandas
        all derived parameters for the run that converged
    """

    # Create dataframe to hold parameters for the run                                      
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    lower_bounds = [nu_range[0], -9, 0.3, 0.5]
    upper_bounds = [nu_range[1], -5, 1.4, 1.0]

    # Apply the LH samples to the sample space
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)

    # Loop through all of the samples and attempt to find a best-fit
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, temps, logged_concs, p0=ps,
                bounds=(lower_bounds,upper_bounds), maxfev=20000)
            # Calculate goodness of fit parameter
            X2 = reduced_chi_squared_test(p, temps, logged_concs, np.ones(len(logged_concs)))
            # Store in the in the dataframe
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            # Do not fail if convergence can't be found.
            if not silent:
                print('Failed to converge with parameters', ps)
    return run_params

In [21]:
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+INP_concs, '_NT')
med_lc = logged_concs.median(axis=1)
med_rc = raw_concs.median(axis=1)

In [22]:
median_params = median_4_param(temps[(temps < -6) & (temps > -25)], med_lc[(temps < -6) & (temps > -25)], med_rc[(temps < -6) & (temps > -25)], samples, 'median')
min_row_index = median_params['X2'].idxmin()

# Now you can access the entire row using iloc
min_row = median_params.iloc[min_row_index]

In [23]:
min_row

nu                                               -4.212565
T_max                                            -6.955521
a                                                 1.271137
b                                                      0.5
X2                                                0.030058
cov      [[0.831195751090417, -0.21035032112416055, -0....
Name: 9, dtype: object

In [24]:
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+nX_vals, '_nS')
med_lc = logged_concs.median(axis=1)
med_rc = raw_concs.median(axis=1)
median_params = median_4_param(temps[(temps < -6) & (temps > -24)], med_lc[(temps < -6) & (temps > -24)], med_rc[(temps < -6) & (temps > -24)], samples, 'median', nu_range=(12, 17))
min_row_index = median_params['X2'].idxmin()

# Now you can access the entire row using iloc
min_row = median_params.iloc[min_row_index]

In [25]:
export_params

,nu_nN,T_max_nN,a_nN,b_nN,nu_nS,T_max_nS,a_nS,b_nS,nu_nV,T_max_nV,a_nV,b_nV,nu_n_INP,T_max_n_INP,a_n_INP,b_n_INP
unique_ID,,,,,,,,,,,,,,,,
c273r1t,-13.696657,-6.959882,1.031066,0.500000,15.554510,-6.959548,1.031338,0.500000,31.045045,-6.958510,1.032167,0.500000,-4.521583,-6.959910,1.031049,0.500000
c273r2t,-15.285553,-12.999198,0.615966,0.793138,12.193049,-13.000000,0.532919,0.836064,25.518428,-12.998465,0.617535,0.792366,-4.459991,-12.999211,0.615950,0.793146
c274r1t,-14.772069,-6.643468,1.050000,0.569511,15.200055,-6.643450,1.050000,0.569513,32.071379,-6.642664,1.050000,0.569605,-3.390939,-6.643481,1.050000,0.569510
c274r3t,-16.999938,-4.000000,0.605890,0.818255,12.172734,-6.000000,0.311117,0.959913,25.288288,-6.000000,0.313467,0.962641,-5.485002,-4.000000,0.605883,0.818258
c275r1t,-16.126016,-7.000000,0.950245,0.543876,13.297670,-7.000000,0.933365,0.548288,27.458321,-7.000000,0.919134,0.555043,-3.695110,-7.000000,0.971554,0.538784
c276r2t,-14.428968,-7.000000,1.026988,0.552503,15.284703,-7.000000,1.029540,0.552131,31.458975,-6.999999,1.033756,0.550940,-2.978206,-7.000000,1.029342,0.552928
c276r4t,-16.269797,-10.000000,1.049500,0.565115,13.043524,-10.000000,1.030780,0.569812,27.702800,-9.999999,1.049661,0.564948,-4.162177,-10.000000,1.032194,0.570259
c277r1t,-16.588036,-8.000000,1.007617,0.622608,13.283459,-8.000000,0.757497,0.705031,27.695984,-8.000000,1.049051,0.611748,-4.080409,-8.000000,0.977644,0.631185
c278r2t,-15.645642,-7.000000,0.887314,0.652976,13.826672,-7.000000,0.890195,0.651678,29.345314,-7.000000,0.890330,0.651606,-3.136699,-7.000000,0.885119,0.653918


In [26]:
min_row

nu                                               12.824831
T_max                                            -6.748666
a                                                 1.273831
b                                                      0.5
X2                                                0.034164
cov      [[3.278027627309631, -1.584442619410105, -2.04...
Name: 1, dtype: object